In [8]:
# Install all necessary libraries
!pip install -q "transformers>=4.41.2" "datasets" "accelerate" "bitsandbytes>=0.43.2" "peft" "trl"

!mkdir -p /usr/local/lib/python3.11/dist-packages/trl/templates/
!touch /usr/local/lib/python3.11/dist-packages/trl/templates/lm_model_card.md

import torch
from datasets import load_dataset, concatenate_datasets
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer, SFTConfig
import json
import os

# --- Dataset 1: MedQuAD (Medical Question Answering) ---
print("Downloading MedQuAD dataset...")
medquad_dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset", split="train")

def format_medquad(example):
    return {
        "instruction": example["Question"],
        "output": example["Answer"]
    }
medquad_dataset = medquad_dataset.map(format_medquad, remove_columns=["Question", "Answer"])
print(f"Loaded {len(medquad_dataset)} examples from MedQuAD.")


# --- Dataset 2: AI Medical Chatbot (Conversational) ---
print("\nDownloading AI Medical Chatbot conversational dataset...")
medical_chatbot_dataset = load_dataset("ruslanmv/ai-medical-chatbot", split="train")

def format_chatbot_dataset(example):
    instruction = f"Patient Description: {example['Description']}\n\nPatient Dialogue: {example['Patient']}"
    return {
        "instruction": instruction,
        "output": example["Doctor"]
    }
medical_chatbot_dataset = medical_chatbot_dataset.map(format_chatbot_dataset, remove_columns=["Description", "Patient", "Doctor"])
print(f"Loaded {len(medical_chatbot_dataset)} examples from AI Medical Chatbot.")


# --- Combine the datasets ---
print("\nCombining datasets...")
combined_dataset = concatenate_datasets([
    medquad_dataset.select(range(1000)),
    medical_chatbot_dataset.select(range(1000))
]).shuffle(seed=42)

print("\nCombined dataset created successfully:")
print(combined_dataset)
print(f"\nExample from combined set: {combined_dataset[0]}")

# Load Model and Tokenizer with Quantization
model_id = "microsoft/Phi-3-mini-4k-instruct"
# This configures 4-bit quantization to save memory
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
# Load the model with the specified quantization config
print("\nLoading base model...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
print("Model and tokenizer loaded.")

# PEFT/LoRA Configuration
model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["qkv_proj", "o_proj", "gate_up_proj", "down_proj"]
)
peft_model = get_peft_model(model, lora_config)

# Setup Training Configuration using SFTConfig
training_args = SFTConfig(
    output_dir="./phi3_finetuned_results",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=10,
    fp16=True,
    max_seq_length=1024,
    packing=True,
    report_to="none",
)

# Create and Run the SFT Trainer
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"<|user|>\n{example['instruction'][i]}<|end|>\n<|assistant|>\n{example['output'][i]}<|end|>"
        output_texts.append(text)
    return output_texts

trainer = SFTTrainer(
    model=peft_model,
    train_dataset=combined_dataset,
    peft_config=lora_config,
    args=training_args,
    formatting_func=formatting_prompts_func,
)

print("\nStarting fine-tuning on combined dataset...")
trainer.train()
print("Fine-tuning completed.")

# Save the Fine-tuned Model for Download
output_model_dir = "./phi3_finetuned_model"
print(f"Saving fine-tuned model to {output_model_dir}")
trainer.save_model(output_model_dir)

print("Model saved successfully. Zipping the model for download...")
!zip -r phi3_finetuned_model.zip phi3_finetuned_model


Map:   0%|          | 0/16407 [00:00<?, ? examples/s]

Loaded 16407 examples from MedQuAD.



Map:   0%|          | 0/256916 [00:00<?, ? examples/s]

Loaded 256916 examples from AI Medical Chatbot.

Combining datasets...

Combined dataset created successfully:
Dataset({
    features: ['qtype', 'instruction', 'output'],
    num_rows: 2000
})

Example from combined set: {'qtype': None, 'instruction': 'Patient Description: Q. Why are there abdominal pain with loose motion and fever?\n\nPatient Dialogue: Hello doctor, There is abdomen pain since three days. Pain is not continous but really painful. Little loose motion, light headache when gets out of bed, little fever on day 1 and 2.', 'output': 'Hi. Such pain in abdomen with stool discomfort can happen as a result of stomach infection. Important here is clinical examination to know the site of the pain. So it is advisable to see your doctor. If he would feel suspicious, he would send you for an ultrasound abdomen. If all normal, you would be treated with antibiotics like Ofloxacin. Till you get medical help, you may take a combination of Pantoprazole and Domperidone that would reduce y

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...


Starting fine-tuning on combined dataset...


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.730800
20,1.558500
30,1.544600
40,1.463000
50,1.441200
60,1.470000


Repo card metadata block was not found. Setting CardData to empty.


Fine-tuning completed.
Saving fine-tuned model to ./phi3_finetuned_model
Model saved successfully. Zipping the model for download...
  adding: phi3_finetuned_model/ (stored 0%)
  adding: phi3_finetuned_model/tokenizer_config.json (deflated 86%)
  adding: phi3_finetuned_model/tokenizer.json (deflated 85%)
  adding: phi3_finetuned_model/adapter_model.safetensors (deflated 7%)
  adding: phi3_finetuned_model/adapter_config.json (deflated 55%)
  adding: phi3_finetuned_model/README.md (deflated 65%)
  adding: phi3_finetuned_model/chat_template.jinja (deflated 60%)
  adding: phi3_finetuned_model/added_tokens.json (deflated 62%)
  adding: phi3_finetuned_model/special_tokens_map.json (deflated 79%)
  adding: phi3_finetuned_model/training_args.bin (deflated 51%)
  adding: phi3_finetuned_model/tokenizer.model (deflated 55%)
